In [80]:
from pkdtools import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mpl_toolkits.mplot3d import Axes3D  # not always necessary but good for older versions
import imageio
import os
import seaborn as sns
import pandas as pd


G = 6.67430e-11  # m^3 kg^-1 s^-2
yr = 3.15576e7  # seconds
au = 1.496e11  # m
M_sun = 1.989e30  # kg

# Center body parameters
r_center = 3.3895e6  # m: Martian radius
M_center = 6.39e23 # kg: Martian mass
rho_center = M_center / (4/3 * np.pi * r_center**3)  # kg/m^3: Martian density

# 
rho = 1e3
tau = 0.3   # optical depth
r = 0.5    # 0.25 - 0.75

R_roche = 1.05 * (M_center/rho)**(1/3)

Sigma = 4 * rho * r * tau / 3
orbit_radius = 1.8 * r_center      # Roche limit is a bit larger: 2.5 r_center
orbit_speed = np.sqrt(G * M_center / orbit_radius)
omega_orb = np.sqrt(G * M_center / (orbit_radius)**3)
period_orb = 2 * np.pi / omega_orb
escape_speed = np.sqrt(2 * G * 4/3 * rho * r**3 / r)
unit_speed = omega_orb * r


lambda_crit = 4 * np.pi**2 * G * Sigma / omega_orb**2
Lx = 5 * lambda_crit
Ly = 5 * lambda_crit
Lz = 5 * r   # 5 m
timestep = 9.9e-8 * yr / 2 / np.pi  

theoretical_collision_speed = Lz / orbit_radius * orbit_speed

print('Oribit Period (h): ', period_orb/3600)
print('timestep (h): ', 1e-07 * yr/2/np.pi/3600)
print('orbital radius: ', orbit_radius, orbit_radius/ R_roche)
print('orbital speed (m/s): ', orbit_speed)
print('lambda crit (m): ', lambda_crit)
print('Surface density (kg/m^2): ', Sigma)
print('Scale height: ', Lz / orbit_radius)
print('Unit speed (m/s): ', unit_speed)
print('Escape speed: ', escape_speed/unit_speed)

print('Theoretical collision speed: ', theoretical_collision_speed/unit_speed)
print('dimensions: ', Lx, Ly, Lz)

Oribit Period (h):  4.027506886762685
timestep (h):  0.00013951522311435544
orbital radius:  6101100.0 0.6746086283130668
orbital speed (m/s):  2643.9255347622316
lambda crit (m):  2.806165650644975
Surface density (kg/m^2):  200.0
Scale height:  8.195243480683811e-07
Unit speed (m/s):  0.00021667613502173637
Escape speed:  0.9735228557834338
Theoretical collision speed:  10.000000000000002
dimensions:  28.061656506449747 28.061656506449747 5.0


In [22]:
print('steps for one orbital period', period_orb / yr * 2*np.pi / 9.96e-8)

steps for one orbital period 33946.23326257732


In [81]:
omega_orb

0.00043335227004347274

In [78]:
Lx/au

8.68416286221583e-10

In [79]:


# plot gif of spins
def plot_spin_gif(filenames, plot = 'spin rate', output_path='gif/Mars_ring_spin.gif'):
    images = []

    for idx, filename in enumerate(filenames):
        print(f"Processing frame {idx} of {len(filenames)}")
        Particles = ss_in(filename, units='mks')
        p_spin = np.zeros((len(Particles), 3))
        p_spin_rate = np.zeros(len(Particles))
        obliquity = np.zeros(len(Particles))
        for i in range(len(Particles)):
            p_spin[i] = np.array([Particles[i].wx, Particles[i].wy, Particles[i].wz + omega_orb ])
            p_spin_rate[i] = np.linalg.norm(p_spin[i])
            obliquity[i] = np.arccos(p_spin[i, 2] / np.linalg.norm(p_spin[i])) * 180/np.pi

        
        
        if plot == 'spin rate':
            fig = plt.figure()
            plt.title(f'Frame {idx}')
            plt.hist(p_spin_rate/omega_orb, bins=30)
            plt.xlabel('Spin Rate')
            plt.ylabel('Count')

        elif plot == 'obliquity':
            fig = plt.figure()
            plt.title(f'Frame {idx}')
            plt.hist(obliquity, bins=30)
            plt.xlabel('Obliquity')
            plt.ylabel('Count')

        elif plot == 'distribution':
                # Create the figure
            fig = plt.figure(figsize=(8, 8))
            gs = gridspec.GridSpec(4, 4)

            # Main scatter plot
            ax_main = fig.add_subplot(gs[1:4, 0:3])
            ax_main.scatter(obliquity, p_spin_rate/omega_orb,  alpha=0.5)
            ax_main.set_xlim([0, 180])
            ax_main.set_ylim([0, 10])
            ax_main.set_xlabel('Obliquity')
            ax_main.set_ylabel('Spin Rate')
            ax_main.set_title(f'Frame {idx}')

            # Histogram on the top
            ax_top = fig.add_subplot(gs[0, 0:3], sharex=ax_main)
            ax_top.hist(obliquity, bins=20, alpha=0.6)
            ax_top.axis('off')  # Optional: hide axes if you want a cleaner look

            # Histogram on the right
            ax_right = fig.add_subplot(gs[1:4, 3], sharey=ax_main)
            ax_right.hist(p_spin_rate/omega_orb, bins=20, orientation='horizontal', alpha=0.6)
            ax_right.axis('off')  # Optional

            plt.tight_layout()

            

        # Save the frame to a temporary image
        temp_filename = f'_frame_{idx}.png'
        plt.savefig(temp_filename, dpi=100)
        plt.close(fig)

        images.append(imageio.imread(temp_filename))
        os.remove(f'_frame_{idx}.png')

    # Save the images as a GIF
    imageio.mimsave(output_path, images)

    # Clean up temp files
    # for idx in range(len(filenames)):
    #     os.remove(f'_frame_{idx}.png')

    print(f"GIF saved to {output_path}")

start_index = 0
end_index = 300000
filenames = ['results/tau_0_3_R_1_8/ss.' + str(i).zfill(10) for i in range(start_index, end_index, 10000)]

print(filenames)


plot_spin_gif(filenames, plot = 'distribution', output_path='gif/Spin_tau_0_3_R_1_8.gif')

['results/tau_0_3_R_1_8/ss.0000000000', 'results/tau_0_3_R_1_8/ss.0000010000', 'results/tau_0_3_R_1_8/ss.0000020000', 'results/tau_0_3_R_1_8/ss.0000030000', 'results/tau_0_3_R_1_8/ss.0000040000', 'results/tau_0_3_R_1_8/ss.0000050000', 'results/tau_0_3_R_1_8/ss.0000060000', 'results/tau_0_3_R_1_8/ss.0000070000', 'results/tau_0_3_R_1_8/ss.0000080000', 'results/tau_0_3_R_1_8/ss.0000090000', 'results/tau_0_3_R_1_8/ss.0000100000', 'results/tau_0_3_R_1_8/ss.0000110000', 'results/tau_0_3_R_1_8/ss.0000120000', 'results/tau_0_3_R_1_8/ss.0000130000', 'results/tau_0_3_R_1_8/ss.0000140000', 'results/tau_0_3_R_1_8/ss.0000150000', 'results/tau_0_3_R_1_8/ss.0000160000', 'results/tau_0_3_R_1_8/ss.0000170000', 'results/tau_0_3_R_1_8/ss.0000180000', 'results/tau_0_3_R_1_8/ss.0000190000', 'results/tau_0_3_R_1_8/ss.0000200000', 'results/tau_0_3_R_1_8/ss.0000210000', 'results/tau_0_3_R_1_8/ss.0000220000', 'results/tau_0_3_R_1_8/ss.0000230000', 'results/tau_0_3_R_1_8/ss.0000240000', 'results/tau_0_3_R_1_8/s

/var/folders/yx/n0ldhkpn0hq62r679b3mq7tm0000gn/T/ipykernel_9417/354545512.py:65: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(temp_filename))


Processing frame 1 of 30
Processing frame 2 of 30
Processing frame 3 of 30
Processing frame 4 of 30
Processing frame 5 of 30
Processing frame 6 of 30
Processing frame 7 of 30
Processing frame 8 of 30
Processing frame 9 of 30
Processing frame 10 of 30
Processing frame 11 of 30
Processing frame 12 of 30
Processing frame 13 of 30
Processing frame 14 of 30
Processing frame 15 of 30
Processing frame 16 of 30
Processing frame 17 of 30
Processing frame 18 of 30
Processing frame 19 of 30
Processing frame 20 of 30
Processing frame 21 of 30
Processing frame 22 of 30
Processing frame 23 of 30
Processing frame 24 of 30
Processing frame 25 of 30
Processing frame 26 of 30
Processing frame 27 of 30
Processing frame 28 of 30
Processing frame 29 of 30
GIF saved to gif/Spin_tau_0_3_R_1_8.gif


In [50]:
# read data use ssio

# import ssio

# test = ssio.read_SS('results/initcond.ss')[0]
# test[3] * au


# read data use pkdtools

def read_information(filename):
    Particles = ss_in(filename, units='mks')
    p_spin = np.zeros((len(Particles), 3))
    p_spin_rate = np.zeros(len(Particles))
    obliquity = np.zeros(len(Particles))
    p_xyz = np.zeros((len(Particles), 3))
    size = np.zeros(len(Particles))
    for i in range(len(Particles)):
        size[i] = Particles[i].R
        p_xyz[i] = np.array([Particles[i].x, Particles[i].y, Particles[i].z])
        p_spin[i] = np.array([Particles[i].wx, Particles[i].wy, Particles[i].wz + omega_orb ])
        p_spin_rate[i] = np.linalg.norm(p_spin[i])
        obliquity[i] = np.arccos(p_spin[i, 2] / np.linalg.norm(p_spin[i])) * 180/np.pi
    
    # Create a DataFrame
    data = {
        'r': size,
        'x': p_xyz[:, 0],
        'y': p_xyz[:, 1],
        'z': p_xyz[:, 2],
        'spin_x': p_spin[:, 0],
        'spin_y': p_spin[:, 1],
        'spin_z': p_spin[:, 2],
        'obliquity': obliquity
    }

    return pd.DataFrame(data)

initcond = read_information('results/ss.0000000000')

# save the dataframe to a CSV file
initcond.to_csv('initcond.csv', index=False)

In [41]:
Particles = ss_in('results/initcond.ss', units='mks')

In [48]:
Particles[0].R

0.2501467136336691

array([0.25015027409475615, 0.2502972484225756, 0.25044448211690395,
       0.2505919759414825, 0.2507397306110057, 0.25088774705213385,
       0.2510360258815177, 0.2511845679278077, 0.25133337386553306,
       0.25148244468769854, 0.2516317810755115, 0.25178138392411137,
       0.25193125381500053, 0.25208139164972626, 0.25223179822764397,
       0.2523824743514515, 0.25253342082720737, 0.25268463825065507,
       0.25283612786163157, 0.25298789026307583, 0.25313992627467097,
       0.25329223671955253, 0.2534448224243269, 0.25359768443510916,
       0.2537508229374487, 0.2539042396333374, 0.2540579344970284,
       0.2542119092407693, 0.25436616384281413, 0.25452070002576316,
       0.25467551777193076, 0.2548306188143682, 0.25498600313951003,
       0.2551416724909581, 0.25529762685932833, 0.25545386755727995,
       0.25561039590445306, 0.2557672118978523, 0.2559243173078985,
       0.25608171213593456, 0.2562393981632896, 0.2563973753957091,
       0.25655564562553557, 0.25671420